In [1]:
import sys
lib_path = '/home/jovyan/libs'
sys.path.insert(0, lib_path)

In [11]:
%reload_ext autoreload
%autoreload 2

import torch
import whisper
import tqdm

from torcheval.metrics.functional import bleu_score
from datasets import load_from_disk
from utils import audio, attacks, gradient

# Load stuffs

In [3]:
data = load_from_disk("../tedlium/train.hf").with_format("torch").select(range(200))

Loading dataset from disk:   0%|          | 0/109 [00:00<?, ?it/s]

In [4]:
snippets = torch.load("tensors/snippets.pt", weights_only=True).detach()
attacker = snippets[-1, :].unsqueeze(0)
baseline_attacker = torch.tensor([]).unsqueeze(0)

In [5]:
MODEL_NAME = "tiny.en"

model = whisper.load_model(MODEL_NAME).cuda()
model.eval();

/home/jovyan/libs/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [29]:
defender = attacks.PrepareFrontLowpass(cutoff=7500)
baseline_defender = attacks.PrepareFront()

In [30]:
gradient.evaluate(model, attacker, defender, data, clamp_ep=0.005, position=0)

Clamp: 0.005
Prepare Method: prepare_front_lowpass
Snippet Size: (1, 10240)
Position: 0


Inference: 100%|██████████| 200/200 [00:21<00:00,  9.31it/s, Valid Examples: 200 | Empty Sequences: 199 | Total SL: 187 | Non-empty ASL: 187.0 | Total Bleu Score: 0.6026419401168823]



Total valid examples: 200
Success rate (Empty): 0.995
Success rate (ASL): 0.935 (attacked) out of 181.575 (original)
Average Bleu Score: 0.0030132096726447344
Average WER: 0.9969230769230769


In [20]:
gradient.evaluate(model, baseline_attacker, defender, data, clamp_ep=0.005, position=0)

Clamp: 0.005
Prepare Method: prepare_front_lowpass
Snippet Size: (1, 10240)
Position: 0


Inference: 100%|██████████| 200/200 [01:05<00:00,  3.05it/s, Valid Examples: 200 | Empty Sequences: 1 | Total SL: 25148 | Non-empty ASL: 126.37185929648241 | Total Bleu Score: 85.53936767578125] 



Total valid examples: 200
Success rate (Empty): 0.005
Success rate (ASL): 125.74 (attacked) out of 181.575 (original)
Average Bleu Score: 0.4276968240737915
Average WER: 0.5628361985541654
